In [33]:
function sis_triang_sup(A::Matrix{Float64}, b::Vector{Float64})
    X = zeros(length(b))
    for i in length(b):-1:1
        x = (b[i] - sum(X .* A[i, :])) / A[i, i]
        X[i] = x
    end
    return round.(X, digits=5)
end

sis_triang_sup (generic function with 1 method)

In [35]:
A = [5 -2 6 1; 0 3 7 -4; 0 0 4 5; 0 0 0 2]

4×4 Matrix{Int64}:
 5  -2  6   1
 0   3  7  -4
 0   0  4   5
 0   0  0   2

In [36]:
convert.(Float32, A)

4×4 Matrix{Float32}:
 5.0  -2.0  6.0   1.0
 0.0   3.0  7.0  -4.0
 0.0   0.0  4.0   5.0
 0.0   0.0  0.0   2.0

In [41]:
b = [1; -2; 28; 8]

4-element Vector{Int64}:
  1
 -2
 28
  8

In [12]:
sis_triang_sup(A, b)

4-element Vector{Float64}:
 -3.0
  0.0
  2.0
  4.0

In [21]:
function eliminacao_gauss(A::Matrix{Float64}, b::Vector{Float64})

    M = hcat(A, b)
    
    function pivotamento(c::Int64, l::Int64, M::Matrix{Float64})
        # c para coluna atual e l para linha atual
        vetor = abs.(M[l:end, c])
        max = argmax(vetor)
        max_index = max + l - 1
        M[max_index, :], M[l, :] = M[l, :], M[max_index, :]    
    end
    
    for i in 1:length(M[:, 1])
        pivotamento(i, i, M)
        pivot_line = M[i, :]
        for j in (i+1):(length(M[1, :])-1)
            m = M[j, i]/pivot_line[i]
            M[j, :] -= m * pivot_line
        end
    end

    return M[:, 1:(end-1)], M[:, end]
    
end

eliminacao_gauss (generic function with 2 methods)

In [43]:
M = [2. 3. -1.; 4. 4. -3.; 2. -3. 1.]

3×3 Matrix{Float64}:
 2.0   3.0  -1.0
 4.0   4.0  -3.0
 2.0  -3.0   1.0

In [44]:
N = [5.; 3.; -1.]

3-element Vector{Float64}:
  5.0
  3.0
 -1.0

In [22]:
V = eliminacao_gauss(M, N)

([4.0 4.0 -3.0; 0.0 -5.0 2.5; 0.0 0.0 1.0], [3.0, -2.5, 3.0])

In [23]:
V[1]

3×3 Matrix{Float64}:
 4.0   4.0  -3.0
 0.0  -5.0   2.5
 0.0   0.0   1.0

In [24]:
V[2]

3-element Vector{Float64}:
  3.0
 -2.5
  3.0

In [25]:
X = sis_triang_sup(V[1], V[2])

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [26]:
inv(M) * N

3-element Vector{Float64}:
 0.9999999999999998
 2.0
 3.0

In [38]:
A1 = [1. 2. 3.; -3. 2. 1.; 1. -1. 1.]
b1 = [6.; 0.; 1.];

In [30]:
R = eliminacao_gauss(A1, b1)

([-3.0 2.0 1.0; 0.0 2.6666666666666665 3.3333333333333335; 0.0 0.0 1.75], [0.0, 6.0, 1.7500000000000002])

In [31]:
R[1]

3×3 Matrix{Float64}:
 -3.0  2.0      1.0
  0.0  2.66667  3.33333
  0.0  0.0      1.75

In [34]:
sis_triang_sup(R[1], R[2])

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [37]:
function resolver_sistema(A::Matrix{Float64}, b::Vector{Float64})
    I = eliminacao_gauss(A, b)
    return sis_triang_sup(I[1], I[2])
end

resolver_sistema (generic function with 1 method)

In [45]:
resolver_sistema(M, N)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [107]:
A = rand(1:100, (6,6))
A = convert.(Float64, A)

6×6 Matrix{Float64}:
 78.0  24.0  82.0  61.0  44.0  25.0
 17.0  59.0  42.0  72.0   9.0  21.0
 79.0  91.0  48.0  46.0  42.0  52.0
 53.0  14.0  20.0  98.0  92.0  88.0
 56.0  88.0  97.0  10.0  89.0  46.0
  5.0  55.0  76.0  32.0  45.0  73.0

In [50]:
b = rand(1:100, 6)
b = convert.(Float64, b)

6-element Vector{Float64}:
 32.0
 37.0
 22.0
 40.0
 32.0
 78.0

In [52]:
using BenchmarkTools

In [56]:
function teste(n::Int64, range) 
    A = rand(range, (n,n))
    A = convert.(Float64, A)
    b = rand(range, n)
    b = convert.(Float64, b)
    return resolver_sistema(A, b)
end

teste (generic function with 1 method)

In [60]:
t1 = @benchmark teste(8, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.000 μs …  1.498 ms  ┊ GC (min … max): 0.00% … 97.33%
 Time  (median):     14.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.482 μs ± 29.091 μs  ┊ GC (mean ± σ):  3.21% ±  1.95%

  ▄▇█▆▅▁▁▁▁                      ▁▂▂▂▁▂▂▂▁▂▁▂▁▁▁▁ ▁           ▂
  █████████▇▆▆▅▄▅▅▅▃▄▄▅▄▄▄▄▄▅▅▆▇███████████████████████▇▇▆▇▇▄ █
  13 μs        Histogram: log(frequency) by time      33.6 μs <

 Memory estimate: 21.09 KiB, allocs estimate: 158.

In [61]:
t2 = @benchmark teste(16, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  44.300 μs …  1.307 ms  ┊ GC (min … max): 0.00% … 94.32%
 Time  (median):     46.300 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.200 μs ± 52.964 μs  ┊ GC (mean ± σ):  4.51% ±  4.39%

  ▅█▇▅▃▁▁▁                                             ▁▁▁▁   ▂
  █████████▇▇▇▆▇▆▅▅▅▅▅▅▄▅▅▅▆▅▄▄▃▄▃▁▁▃▄▅█▆▆▆▄▃▁▃▁▁▃▄▅▅▇██████▇ █
  44.3 μs      Histogram: log(frequency) by time       109 μs <

 Memory estimate: 99.25 KiB, allocs estimate: 498.

In [64]:
ratio(mean(t2), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             2.9860059296326615
  gctime:           4.195522254928196
  memory:           4.705185185185186
  allocs:           3.151898734177215

In [65]:
t3 = @benchmark teste(32, 10:30)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  188.200 μs …  1.319 ms  ┊ GC (min … max): 0.00% … 62.75%
 Time  (median):     193.700 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   208.273 μs ± 89.915 μs  ┊ GC (mean ± σ):  4.56% ±  8.72%

  █▄▃▁                                                         ▁
  ████▇▅▅▅▅▅▄▃▃▅▆▅▃▁▁▁▁▁▁▁▄▇▆▆▆▄▃▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▅▇ █
  188 μs        Histogram: log(frequency) by time       902 μs <

 Memory estimate: 598.00 KiB, allocs estimate: 1754.

In [66]:
ratio(mean(t3), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             11.913796091796618
  gctime:           16.937311568994737
  memory:           28.34962962962963
  allocs:           11.10126582278481

In [81]:
2 ^ (1.789)

3.4557527496011087

In [80]:
4 ^ (1.789)

11.942227066375622

In [83]:
t4 = @benchmark teste(64, 10:30)

BenchmarkTools.Trial: 4683 samples with 1 evaluation.
 Range (min … max):  970.300 μs …   5.478 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     994.700 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.063 ms ± 236.193 μs  ┊ GC (mean ± σ):  4.83% ± 10.34%

  ▅█▇▅▄▂▂                                          ▁▁▁▁▁▁       ▁
  ████████▇▆▇▆▅▅▃▅▅▃▃▃▁▁▄▁▁▃▄▁▁▄▁▄▁▁▁▁▁▁▃▃▁▁▁▃▃▅▆▇████████▇▇▆▆▆ █
  970 μs        Histogram: log(frequency) by time       1.72 ms <

 Memory estimate: 3.70 MiB, allocs estimate: 6574.

In [84]:
ratio(mean(t4), mean(t1))

BenchmarkTools.TrialRatio: 
  time:             60.77976150843191
  gctime:           91.5350880275624
  memory:           179.71555555555557
  allocs:           41.607594936708864

In [89]:
8 ^ (1.975)

60.75789574083324

In [91]:
using LinearAlgebra

function decompLU(A::Matrix{Float64})
    U = copy(A)
    n = length(A[1,:])
    L = zeros(n, n)
    L[diagind(A)] .= 1   

    for i in 1:n
        pivotamento(i, i, U)
        pivot_line = U[i, :]
        for j in (i+1):n
            m = U[j, i]/pivot_line[i]
            L[j, i] = m
            U[j, :] -= m * pivot_line
        end
    end 
    return L, U
end


decompLU (generic function with 1 method)

In [75]:
X = decompLU(A)

([1.0 0.0 … 0.0 0.0; 3.0 1.0 … 0.0 0.0; … ; 6.357142857142857 2.039442039442039 … 1.0 0.0; 4.5 1.4208754208754208 … 2.3499663424500894 1.0], [14.0 100.0 … 68.0 28.0; 0.0 -297.0 … -137.0 -48.0; … ; 0.0 0.0 … 259.3752590355107 -273.09053071546555; 0.0 0.0 … 0.0 157.76628202313913])

In [96]:
A

6×6 Matrix{Float64}:
 14.0  100.0  80.0  84.0  68.0  28.0
 42.0    3.0  44.0  79.0  67.0  36.0
 43.0   15.0  34.0  31.0   7.0  82.0
  2.0   79.0  61.0  79.0   9.0  39.0
 89.0   30.0  54.0  49.0  12.0  83.0
 63.0   28.0  16.0   4.0  55.0  41.0

In [77]:
X[1]

6×6 Matrix{Float64}:
 1.0        0.0        0.0       0.0      0.0      0.0
 3.0        1.0        0.0       0.0      0.0      0.0
 3.07143    0.983646   1.0       0.0      0.0      0.0
 0.142857  -0.217893  -0.362816  1.0      0.0      0.0
 6.35714    2.03944    2.89861   3.74737  1.0      0.0
 4.5        1.42088    3.46245   7.89552  2.34997  1.0

In [78]:
X[2]

6×6 Matrix{Float64}:
 14.0   100.0    80.0       84.0        68.0       28.0
  0.0  -297.0  -196.0     -173.0      -137.0      -48.0
  0.0     0.0   -18.9197   -56.8292    -67.0976    43.215
  0.0     0.0     0.0        8.68592   -54.9097    40.2202
  0.0     0.0     0.0        0.0       259.375   -273.091
  0.0     0.0     0.0        0.0         0.0      157.766

In [79]:
X[1] * X[2]

6×6 Matrix{Float64}:
 14.0  100.0  80.0  84.0  68.0  28.0
 42.0    3.0  44.0  79.0  67.0  36.0
 43.0   15.0  34.0  31.0   7.0  82.0
  2.0   79.0  61.0  79.0   9.0  39.0
 89.0   30.0  54.0  49.0  12.0  83.0
 63.0   28.0  16.0   4.0  55.0  41.0

In [83]:
function pivotamento(c::Int64, l::Int64, M::Matrix{Float64})
    # c para coluna atual e l para linha atual
    vetor = abs.(M[l:end, c])
    max = argmax(vetor)
    max_index = max + l - 1
    M[max_index, :], M[l, :] = M[l, :], M[max_index, :]    
end

pivotamento (generic function with 1 method)

In [103]:
#pivotamento(1, 1, A)

([89.0, 30.0, 54.0, 49.0, 12.0, 83.0], [89.0, 30.0, 54.0, 49.0, 12.0, 83.0])

In [109]:
A

6×6 Matrix{Float64}:
 78.0  24.0  82.0  61.0  44.0  25.0
 17.0  59.0  42.0  72.0   9.0  21.0
 79.0  91.0  48.0  46.0  42.0  52.0
 53.0  14.0  20.0  98.0  92.0  88.0
 56.0  88.0  97.0  10.0  89.0  46.0
  5.0  55.0  76.0  32.0  45.0  73.0

In [110]:
X = decompLU(A)

([1.0 0.0 … 0.0 0.0; 0.21518987341772153 1.0 … 0.0 0.0; … ; 0.7088607594936709 -0.3567858515955401 … 1.0 0.0; 0.06329113924050633 -0.7477893118031526 … -0.9681042333517418 1.0], [79.0 91.0 … 42.0 52.0; 0.0 -65.84810126582279 … 2.5316455696202524 -26.341772151898738; … ; 0.0 0.0 … 79.45523467752346 22.820387228944988; 0.0 0.0 … 0.0 -73.77788530685758])

In [111]:
X[1]*X[2]

6×6 Matrix{Float64}:
 79.0   91.0      48.0     46.0     42.0      52.0
 17.0  -46.2658   44.9367  25.481   11.5696  -15.1519
 78.0  129.266   125.517   76.8308  84.1878  117.121
 53.0   14.0      20.0     98.0     92.0      88.0
 56.0   88.0      97.0     10.0     89.0      46.0
  5.0   55.0      29.5463  62.6882   2.2426   17.0308

In [112]:
A - X[1]*X[2]

6×6 Matrix{Float64}:
 -1.0  -67.0           34.0          …    2.0          -27.0
  0.0  105.266         -2.93671          -2.56962       36.1519
  1.0  -38.2658       -77.517           -42.1878       -65.1211
  0.0    1.77636e-15   -3.55271e-15       0.0            1.42109e-14
  0.0    0.0            0.0               1.42109e-14    0.0
  0.0    0.0           46.4537       …   42.7574        55.9692

In [113]:
X[1]

6×6 Matrix{Float64}:
 1.0         0.0        0.0        0.0        0.0       0.0
 0.21519     1.0        0.0        0.0        0.0       0.0
 0.987342   -0.598616   1.0        0.0        0.0       0.0
 0.670886    0.714533  -0.373638   1.0        0.0       0.0
 0.708861   -0.356786   0.762052  -0.675227   1.0       0.0
 0.0632911  -0.747789   0.530017   0.699669  -0.968104  1.0

In [114]:
X[2]

6×6 Matrix{Float64}:
 79.0   91.0     48.0          46.0     42.0       52.0
  0.0  -65.8481  34.6076       15.5823   2.53165  -26.3418
  0.0    0.0     98.8412       40.7409  44.2349    50.0108
  0.0    0.0      0.0          71.2275  78.5417    90.6219
  0.0    0.0      0.0           0.0     79.4552    22.8204
  0.0    0.0     -7.10543e-15   0.0      0.0      -73.7779

In [136]:
atol=1e-15
isapprox(abs(det(A)),det(X[2]), rtol=0.000) 

true

In [132]:
det(A)/det(X[2])

-1.0000000000000004

In [131]:
isapprox(3,3.1, rtol=1) 

true

In [133]:
det(A)

-2.1468658680600006e11

In [134]:
det(X[2])

2.1468658680599997e11